In [255]:
import pandas as pd
pd.set_option('display.float_format', '{:,.2f}'.format)
import numpy as np
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

Информация по талицам:

contractor_test - информация о контрагентах 

 - ITN - ИНН
 - name_contractor - название компании контрагента
 - form - передача форм собственности предприятий

contractor_test - информация о контранках

 - number - номер договора
 - date - дата заключения договора
 - ITN_contractor - ИНН контрагента, с которым заключен Договор
 - our_template	 - наш шаблон Договора (1 - Наш договор; 0 - Договор контрагента)
 - price - стоимость контракта 
 - object - объект строительства
 - gu - гарантийные удержания
 - gp - ген.услуги
 - prepayment - аванс (Доля от стоисоти договора)


In [185]:
# Информация о Контрактах

class WorkWithContract:
    data : pd.DataFrame
    copy: bool
    __instence = None
    
    
    def __new__(cls, *args, **kwargs):
        if cls.__instence == None:
            cls.__instence = super().__new__(cls)
        return  WorkWithContract.__instence
        
    
    def __init__(self, data, copy: bool = False):
        self.data = data
        self.info = {'contract' : 'номер договора',
                'date': 'дата заключения договора в формате DD.MM.YYYY',
                'ITN_contractor' : 'ИНН контрагента, с которым заключен Договор',
                'price' : 'стоимость контракта', 
                'object' : 'объект строительства', 
                'our_template': 'наш шаблон Договора (1 - Наш договор; 0 - Договор контрагента)',
                'gu' : 'гарантийные удержания',
                'gp' : 'ген.услуги',
                'prepayment' : 'аванс (Доля от стоисоти договора)'}
        self.gu = 0.05
        self.gp = 0.02
        self.prepayment = 0.30
        if copy:
            self.data = data.copy()
        else:
            self.data = data
            
    
        
    def check_number(self, other):
        """Проверка на наличие номера Договора в Базе"""
        if self.data.isin({'contract' : other}).any()[0]:
            print(f'Контракт c № {other} уже есть в Базе')
        else:
            print(f'Номер {other} свободен, можно использовать!')
            
            
    def our_template(self):
        pass
        
        
    def add_new(self):
        """Добавление нового контракта в Базу"""
        new_row = {}
        for i in self.data.columns:
            print(f'Введите информацию {self.info[i]}')
            a = input()
            if i == 'date':
                a = datetime.strptime(a, '%d.%m.%Y')
            elif i in ['price', 'gu', 'gp', 'prepayment']:
                a = float(a)
            new_row[i] = a
            if i == 'our_template':
                if a == '1':
                    new_row['gu'] = self.gu
                    new_row['gp'] = self.gp
                    new_row['prepayment'] = self.prepayment
                    print('Размер Гарантийных удержаний и Ген услуг проставлены согласно Стандартного шаблона Договора')
                    break            
        new_row_1 = pd.Series(data=new_row)
        self.data = self.data.append(new_row_1, ignore_index=True)
        print(f'Договор записан')
        return self.data
    

# Можно переносить его в отдельную Базу

    def drop_contract(self, other): 
        """Удаление Договора из базы по номеру Договора"""
        self.data.set_index('contract', inplace = True)
        self.data.drop(other, inplace = True)
        print(f'Договор № {other} удален из базы!!!!')
        self.data.reset_index(inplace=True)
        return self.data
    
    def change_terms(self):
# Внесение изменений 
        pass

    
    def end_work(self):
        """Конец работы с документом и запись новой информации"""
#         today_day = datetime.today().strftime('%d-%m-%Y')
        name_file = 'contract_02-06-2022.csv'
        self.data.to_csv(name_file, index = False)
        print(f'Файл {name_file} записан')
    
    
    def __del__(self):
        cls.__instence = None




In [186]:
# Информация о Контрагентах

class WorkWithContractor():
    data : pd.DataFrame
    __instence = None
    
    
    def __new__(cls, *args, **kwargs):
        if cls.__instence == None:
            cls.__instence = super().__new__(cls)
        return cls.__instence
    
    def __init__(self, data):
        self.data = data #База со сведеньями о Контрагентах
        self.info = {'ITN': 'ИНН контрагента',
                'name_contractor':'название компании контрагента',
                'form' : 'передача форм собственности предприятий'}
    
    def check_contractor(self):
        """Проверка на наличие контрагента в Базе по ИНН"""
        print(f'Введите ИНН контрагента')
        ITN_contractor = input()
        if self.data.isin({'ITN' : [ITN_contractor]}).any()[0]:
            print(f'Контрагент с ИНН {ITN_contractor} уже есть в Базе')
            return self.data[self.data['ITN'] == ITN_contractor]
        else:
            print(f'Контрагентa c ИНН {ITN_contractor} нет в Базе Контрагентов')
            
    def add_new(self):
        """Добавление нового контракта в Базу"""
        new_row = {}
        for i in self.data.columns:
            print(f'Введите информацию {self.info[i]}')
            a = input()
            new_row[i] = a     
        new_row_1 = pd.Series(data=new_row)
        self.data = self.data.append(new_row_1, ignore_index=True)
        print(f'Контрагент записан в Базу')
        return self.data
    
    def drop_contractor(self, other):     
        self.data.set_index('ITN', inplace = True)
        self.data.drop(other, inplace = True)
        print(f'Контрагент {other} удален из базы!!!!')
        self.data.reset_index(inplace=True)
        return self.data
            
    def end_work(self):
        """Конец работы с документом и запись новой информации"""
#         today_day = datetime.today().strftime('%d-%m-%Y')
        name_file = 'contractor_02-06-2022.csv'
        self.data.to_csv(name_file, index = False)
        print(f'Файл {name_file} записан')
        
    def __del__(self):
        cls.__instence = None

In [187]:
data = pd.read_csv('contract_02-06-2022.csv')
data_1 = pd.read_csv('contractor_02-06-2022.csv')

In [188]:
contract = WorkWithContract(data)
contractor = WorkWithContractor(data_1)

In [189]:
contract.data

,contract,date,ITN_contractor,price,object,our_template,gu,gp,prepayment
0,1-TERRA-37,2020-08-21 00:00:00,7806564446,35000000.00,TERRA_1,1,0.05,0.02,0.3
1,1-TERRA-41,2020-09-08 00:00:00,7811060998,46577955.88,TERRA_1,1,0.05,0.02,0.3
2,2-TERRA-51,2020-09-26 00:00:00,7811060998,41838268.08,TERRA_2,1,0.05,0.02,0.3
3,B7NO-4169/4188,2020-12-01 00:00:00,7805223443,63331200.00,TERRA_1,0,0.00,0.00,1.0
4,1-TERRA-45,2020-10-01 00:00:00,7814669700,38317649.45,TERRA_1,1,0.05,0.02,0.3
5,B7NO-3227/3242,2020-12-01 00:00:00,7805223443,51907200.00,TERRA_2,0,0.00,0.00,1.0


In [67]:
contractor.data

,ITN,name_contractor,form
0,7810727284,АКВАПОИНТ,ООО
1,7840516562,АРСЕНАЛ СПБ,ООО
2,7810299920,АРТ-КЛИМАТ,ООО
3,7810582416,ВОЛЬТ СПБ,ООО
4,7813539105,ЗАЛЕЙ ПОЛ,ООО
5,7839409453,ИК АЛЬЯНС,ООО
6,7811583941,СК ПЕГАС,ООО
7,7805223443,ОТИС ЛИФТ,ООО
8,7811060998,БЕККЕР,ООО
9,7806564446,ИСУ,ООО


In [68]:
contractor.add_new()

Введите информацию ИНН контрагента
7814669700
Введите информацию название компании контрагента
РОДЕО СПБ
Введите информацию передача форм собственности предприятий
ООО
Контрагент записан в Базу


,ITN,name_contractor,form
0,7810727284,АКВАПОИНТ,ООО
1,7840516562,АРСЕНАЛ СПБ,ООО
2,7810299920,АРТ-КЛИМАТ,ООО
3,7810582416,ВОЛЬТ СПБ,ООО
4,7813539105,ЗАЛЕЙ ПОЛ,ООО
5,7839409453,ИК АЛЬЯНС,ООО
6,7811583941,СК ПЕГАС,ООО
7,7805223443,ОТИС ЛИФТ,ООО
8,7811060998,БЕККЕР,ООО
9,7806564446,ИСУ,ООО


In [256]:
contract.end_work()
contractor.end_work()

Файл contract_02-06-2022.csv записан
Файл contractor_02-06-2022.csv записан


In [265]:
# Подгружаем выгруженные данные по оплатам и по запроцентованным Подрядчикам работам
pay1 = pd.read_csv('pay_1.csv', sep ='\t') # выгрузка по совершённым оплатам
docs1 = pd.read_csv('docs.csv', sep ='\t') # выгрузка по запроцентованным работам и штрафам

In [266]:
# Превести столбец sum в формат float

def change_str_fl(column):
    column = column.apply(lambda x: x.replace(',', '.').replace(' ', '')).astype(float)
    return column

pay1['sum'] = change_str_fl(pay1['sum'])
docs1['sum'] = change_str_fl(docs1['sum'])

In [267]:
pay1.head()

,id,date_of_pay,sum,ITN,contactor,payment for completed work,contract,object
0,NaN,NaN,"500,000.00",7806564446,ИСУ,1,1-TERRA-37,TERRA_1
1,NaN,NaN,"258,807.93",7806564446,ИСУ,1,1-TERRA-37,TERRA_1
2,NaN,NaN,"1,500,000.00",7806564446,ИСУ,1,1-TERRA-37,TERRA_1
3,NaN,NaN,"344,587.73",7806564446,ИСУ,1,1-TERRA-37,TERRA_1
4,NaN,NaN,"200,000.00",7806564446,ИСУ,1,1-TERRA-37,TERRA_1


In [270]:
# Расчёт задолности по Контрагенту (БЕЗ УЧЁТА ГП и ГУ)
def debs_contractor(pay, docs):
    costs = pay.groupby('contactor').agg({'sum': 'sum'})
    done = docs.groupby('contactor').agg({'sum': 'sum'})
    other = contract.data.groupby('contract').agg({'gu': 'sum', 'gp': 'sum'})
    result = done-costs
    result = result.dropna(axis=0)
    return result

In [271]:
debs_contractor(pay1, docs1)

,sum
contactor,
БЕККЕР,"29,003,022.89"
ИСУ,"8,897,453.57"
ОТИС ЛИФТ,"-45,307,680.00"
РОДЕО СПБ,"5,638,398.40"


In [250]:
other = contract.data.groupby('contract').agg({'gu': 'sum', 'gp': 'sum'})
other.gu

contract
1-TERRA-37       0.05
1-TERRA-41       0.05
1-TERRA-45       0.05
2-TERRA-51       0.05
B7NO-3227/3242   0.00
B7NO-4169/4188   0.00
Name: gu, dtype: float64

In [253]:
# Функция расчёта задолности по Контракту
def debt_cotract(pay, docs):
    costs = pay.groupby(['contactor', 'object', 'contract']).agg({'sum': 'sum'})
    done = docs.groupby(['contactor', 'object', 'contract']).agg({'sum': 'sum'})
    other = contract.data.groupby('contract').agg({'gu': 'sum', 'gp': 'sum'})
    result = done.apply(lambda x: x*(1-other.gu-other.gp))- costs
    result = result.dropna(axis=0)
    return result

In [254]:
debt_cotract(pay=pay1, docs=docs1)

sum
contactor object  contract                    
БЕККЕР    TERRA_1 1-TERRA-41     10,811,688.24
          TERRA_2 2-TERRA-51     13,342,193.41
ИСУ       TERRA_1 1-TERRA-37      7,183,611.99
ОТИС ЛИФТ TERRA_2 B7NO-3227/3242  2,814,720.00
РОДЕО СПБ TERRA_1 1-TERRA-45      3,648,633.33

In [272]:
# Функция расчёта задолности по Объекту

def debt_object(pay, docs):
    costs = pay.groupby('object').agg({'sum': 'sum'})
    done = docs.groupby('object').agg({'sum': 'sum'})
#     other = contract.data.groupby('contract').agg({'gu': 'sum', 'gp': 'sum'})
    result = done- costs
    result = result.dropna(axis=0)
    return result

In [273]:
debt_object(pay1, docs1)

,sum
object,
TERRA_1,"-19,913,675.51"
TERRA_2,"18,144,870.37"


In [ ]:
# Расчёт задолности по Объекту
class ObjectDebt():
    data : pd.DataFrame
    
    def __init__(self, data):
        self.data = data
    
# Расчёт задолности по Контрагенту


class CotractorDebt():

    def __init__(self, data):
        self.data = data

# Расчёт задолности по Контракту

class ContractDebt():
    pass